# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities_weather.csv')
weather_df

city country  max_temp  humidity  cloudiness  wind_speed    lat  \
0       Torbay      CA     32.00        74          90        9.17  47.67   
1      Licheng      CN     73.00        88          90        8.95  23.30   
2       Urzhum      RU      8.67        94          68        6.29  57.11   
3    Etchoropo      MX     66.99        64           0        3.00  26.77   
4      Ushuaia      AR     46.40        70          75       13.87 -54.80   
..         ...     ...       ...       ...         ...         ...    ...   
546      Pelym      RU     18.64        91          98        7.20  61.01   
547   Escanaba      US     24.80        73          90        3.36  45.75   
548     Manaus      BR     89.60        66          40        3.36  -3.10   
549  Burmakino      RU     21.20        73          90        8.95  57.42   
550     Laguna      US     50.00        87          90       11.41  38.42   

        lon        date  
0    -52.73  1607879851  
1    113.83  1607879717  
2     50.00  1607879874  
3   -109.67  1607879874  
4    -68.30  1607879874  
..      ...         ...  
546   62.00  1607879941  
547  -87.06  1607879941  
548  -60.02  1607879941  
549   40.24  1607879942  
550 -121.42  1607879639  

[551 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[["lat", "lon"]]
humidity = weather_df['humidity'].astype(float)

In [4]:
fig = gmaps.figure(center =(weather_df["lat"].mean(),weather_df["lon"].mean()), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_conditions = weather_df.dropna()
narrowed_conditions = narrowed_conditions[narrowed_conditions['max_temp']<80]
narrowed_conditions = narrowed_conditions[narrowed_conditions['max_temp']>70]
narrowed_conditions = narrowed_conditions[narrowed_conditions['wind_speed']<10]
narrowed_conditions = narrowed_conditions[narrowed_conditions['cloudiness']==0]

narrowed_conditions.count()

city          10
country       10
max_temp      10
humidity      10
cloudiness    10
wind_speed    10
lat           10
lon           10
date          10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_conditions[["city","country", "lat", "lon"]]
hotel_df['hotel_name'] = ""
hotel_df

<ipython-input-6-7bf838360e86>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['hotel_name'] = ""


city country    lat     lon hotel_name
35                 Gwadar      PK  25.12   62.33           
45           Kruisfontein      ZA -34.00   24.73           
78                 Avarua      CK -21.21 -159.78           
117             Cape Town      ZA -33.93   18.42           
269         Graaff-Reinet      ZA -32.25   24.53           
290              Tessalit      ML  20.20    1.01           
323                Ormara      PK  25.21   64.64           
391                Safaga      EG  26.73   33.94           
449  Makkah al Mukarramah      SA  21.43   39.83           
477                 Chauk      MM  20.88   94.82

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    coordinates = f"{row['lat']},{row['lon']}"
    params = {"key": g_key,
              "location": coordinates,
              "radius": 5000,
              "keyword": "hotel",
              "type": "lodging",
              "language": "en"
             }
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'hotel_name'] = response['results'][0]['name']
    
    except (KeyError, IndexError):
        hotel_df.loc[index, 'hotel_name'] = "Hotel not found"

/Users/atokarev/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [8]:
hotel_df

city country    lat     lon  \
35                 Gwadar      PK  25.12   62.33   
45           Kruisfontein      ZA -34.00   24.73   
78                 Avarua      CK -21.21 -159.78   
117             Cape Town      ZA -33.93   18.42   
269         Graaff-Reinet      ZA -32.25   24.53   
290              Tessalit      ML  20.20    1.01   
323                Ormara      PK  25.21   64.64   
391                Safaga      EG  26.73   33.94   
449  Makkah al Mukarramah      SA  21.43   39.83   
477                 Chauk      MM  20.88   94.82   

                                    hotel_name  
35               Zaver Pearl Continental Hotel  
45                             Hotel not found  
78   The Rarotongan Beach Resort & Lagoonarium  
117      Atlanticview Cape Town Boutique Hotel  
269                Cypress Cottage Guest House  
290                            Hotel not found  
323            Alqadri hotel zero point ormara  
391                       lagoonie Lodge Hotel  
449                           Tera Sedky Hotel  
477                        Bo Khin Guest House

In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
city_locations = hotel_df[["lat", "lon"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(city_locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))